In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import normalize
from scipy.sparse import hstack

from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics.classification import accuracy_score, log_loss
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import os
import gensim
nltk.download('punkt')

from collections import defaultdict

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Getting the Model

In [0]:
model = gensim.models.Word2Vec.load('/content/gdrive/My Drive/w2vmodel_big')

In [0]:
model.wv.most_similar('mutation')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('mutations', 0.7920259833335876),
 ('substitution', 0.6018621921539307),
 ('case', 0.5685200095176697),
 ('one', 0.551011323928833),
 ('alteration', 0.5437765121459961),
 ('found', 0.5348433256149292),
 ('cases', 0.5271749496459961),
 ('germline', 0.5236974954605103),
 ('somatic', 0.5220752954483032),
 ('patients', 0.5189864039421082)]

### Vocab Size

In [0]:
vocab = list(model.wv.vocab)
print(len(vocab))

225844


In [0]:
from itertools import islice
list(islice(vocab, 13030, 13050))

['d9e',
 'gt239',
 'genetex',
 'bethyl',
 'm2-coupled',
 'horseradish-peroxidase-labeled',
 'glo',
 'x-tremegene',
 'glutathione-coupled',
 'taconic',
 'labs',
 'selleckchem',
 'synthesized',
 'hagst-s6k1',
 'ha-gst-akt1',
 'ha-gst-rheb1',
 'flag-14-3-3',
 'generous',
 'yaffe',
 'hek-293t-trex']

# Getting the Data

In [0]:
w2v_data = pd.read_csv('/content/gdrive/My Drive/w2v_data.csv')

In [0]:
w2v_data.head()

,Text,Class
0,fam58a truncating mutations cyclin-dependent k...,1
1,cbl w802* abstract background non-small cell l...,2
2,cbl q249e abstract background non-small cell l...,2
3,cbl n454d recent evidence demonstrated acquire...,3
4,cbl l399v oncogenic mutations monomeric casita...,4


In [0]:
class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = len(word2vec.wv.syn0[0])

    def fit(self, X):
        return self

    def fit_transform(self, X):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of 
        # known idf's
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])


# Tokenizer and Embedding

### Tokenizer

In [0]:
class MyTokenizer:
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        transformed_X = []
        for document in X:
            tokenized_doc = []
            for sent in nltk.sent_tokenize(document):
                tokenized_doc += nltk.word_tokenize(sent)
            transformed_X.append(np.array(tokenized_doc))
        return np.array(transformed_X)
    
    def fit_transform(self, X, y=None):
        return self.transform(X)

### Embedding

In [0]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(word2vec.wv.syn0[0])

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = MyTokenizer().fit_transform(X)
        
        return np.array([
            np.mean([self.word2vec.wv[w] for w in words if w in self.word2vec.wv]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
    
    def fit_transform(self, X, y=None):
        return self.transform(X)


In [0]:
mean_embedding_vectorizer = MeanEmbeddingVectorizer(model)
mean_embedded = mean_embedding_vectorizer.fit_transform(w2v_data['Text'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  


In [0]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import ComplementNB
from sklearn.linear_model import SGDClassifier

In [0]:
type(mean_embedded)

numpy.ndarray

In [0]:
mean_embedded.shape

(3321, 300)

In [0]:
print(mean_embedded[0])

[-0.33483508 -0.4234247   0.1903168  -0.07608256  0.50291115  0.07554512
  0.15030444 -0.01006396 -0.13573194  0.15463531 -0.231468    0.23079197
 -0.03590651 -0.02440811 -0.07382004  0.07969389  0.03842213  0.13287629
  0.28766048  0.06228872  0.15174717 -0.05815761  0.28997976 -0.43329698
 -0.1056528  -0.09698886 -0.2733862  -0.05692174 -0.00884036 -0.19348675
  0.2821662   0.19719823 -0.24916247  0.04985579 -0.6404954   0.0165292
 -0.57253087  0.06189586  0.04557919  0.07512408 -0.01794662  0.28100377
  0.20604512 -0.11275636  0.5672029  -0.14782198  0.26388898 -0.3960545
  0.23061322 -0.514577   -0.06700831  0.21000037 -0.06305815 -0.23955007
  0.35592777 -0.17854221 -0.195387    0.07227565  0.0394758  -0.10733049
 -0.15965869 -0.29026085 -0.46385333 -0.25229338  0.06003476 -0.18649456
  0.06582275 -0.08651158 -0.3130542  -0.23331259  0.0225482   0.057657
  0.1660362   0.02748245 -0.36863202  0.31750703  0.24861993  0.61338115
 -0.09689697  0.759378    0.15205783 -0.01211227  0.183

# Prediction Function

In [0]:
def predict_and_plot_confusion_matrix(X, Y, clf):
    train_x, test_x, train_y, test_y = train_test_split(X, Y, stratify= Y , test_size=0.2)
    
    
    
    clf.fit(train_x, train_y)
    
    sig_clf = CalibratedClassifierCV(clf, method="sigmoid")
    sig_clf.fit(train_x, train_y)
    pred_y = sig_clf.predict(test_x)

    # for calculating log_loss we will provide the array of probabilities belongs to each class
    print("Log loss :",log_loss(test_y, sig_clf.predict_proba(test_x)))
    print("Actual :", test_y[0])
    print("Probability :",np.round(sig_clf.predict_proba(test_x[0].reshape(1, -1) ),4 ))
    print("Predicted :", sig_clf.predict(test_x[0].reshape(1, -1) ))
    # calculating the number of data points that are misclassified
    # print("Number of mis-classified points :", np.count_nonzero((pred_y- test_y))/test_y.shape[0], 1-np.count_nonzero((pred_y- test_y))/test_y.shape[0])
    print(confusion_matrix(test_y, pred_y))
    print("Score :",sig_clf.score(test_x, test_y))

# Normalization of Encoded Vectors

In [0]:
mean_embedded  = normalize(mean_embedded, axis = 0)

In [0]:
min_val = min( [ min(vec) for vec in mean_embedded] )
print(min_val)

-0.29168472


In [0]:
x, y  = mean_embedded.shape
print(x,y)

3321 300


In [0]:
for i in range(x):
  for j in range(y):
    mean_embedded[i][j] += abs(min_val)

In [0]:
min( [ min(vec) for vec in mean_embedded] )

0.0

In [0]:
mean_embedded = np.array( mean_embedded )
y = np.array( list( w2v_data['Class'].values ) )

In [0]:

y.shape

(3321,)

In [0]:
mean_embedded.shape

(3321, 300)

In [0]:
mean_embedded[0].reshape(1, -1).shape

(1, 300)

# Classification

## Naive Bayes

In [0]:
predict_and_plot_confusion_matrix(mean_embedded, y,
                                 MultinomialNB()
                                 )

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Log loss : 1.4050483613028375
Actual : 5
Probability : [[0.329  0.0301 0.0293 0.3586 0.1126 0.1021 0.0337 0.0037 0.0009]]
Predicted : [4]
[[ 42  13   0  43   0   2  14   0   0]
 [  8  25   0   5   0   0  53   0   0]
 [  1   1   0   8   0   0   8   0   0]
 [ 31   5   0  70   0   3  28   0   0]
 [ 15   2   0   7  10   3  11   0   0]
 [  4   3   0  12   0  15  21   0   0]
 [  4  16   0  10   0   0 161   0   0]
 [  3   0   0   0   0   0   1   0   0]
 [  3   0   0   1   0   0   0   0   3]]
Score : 0.49022556390977445


## XGBoost

In [0]:
predict_and_plot_confusion_matrix(mean_embedded, y,
                                  XGBClassifier(max_depth=4,
                                      objective='multi:softprob',
                                      learning_rate=0.03333,
                                      )
                                 )

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/calibration.py:447: RuntimeWarning: divide by zero encountered in log
  l = -(T * np.log(P + tiny) + T1 * np.log(1. - P + tiny))
/usr/local/lib/python3.6/dist-packages/sklearn/calibration.py:447: RuntimeWarning: divide by zero encountered in log
  l = -(T * np.log(P + tiny) + T1 * np.log(1. - P + tiny))
/usr/local/lib/python3.6/dist-packages/sklearn/calibration.py:447: RuntimeWarning: divide by zero encountered in log
  l = -(T * np.log(P + tiny) + T1 * np.log(1. - P + tiny))
/usr/local/lib/python3.6/dist-packages/sklearn/calibration.py:447: RuntimeWarning: divide by zero encountered in log
  l = -(T * np.log(P + tiny) + T1 * np.log(1. - P + tiny))
/usr/local/lib/python3.

Log loss : 1.1139643611411982
Actual : 2
Probability : [[0.0881 0.3649 0.0209 0.0509 0.0421 0.0388 0.3803 0.0067 0.0073]]
Predicted : [7]
[[ 74   2   0  22   4   2   9   0   1]
 [  5  42   0   0   0   0  44   0   0]
 [  3   0   2   6   1   0   6   0   0]
 [ 26   4   0  84   4   0  19   0   0]
 [ 14   4   0   5  13   4   8   0   0]
 [ 10   1   0   3   3  30   8   0   0]
 [  3  22   1   5   1   0 159   0   0]
 [  2   1   0   0   0   0   1   0   0]
 [  1   0   0   0   0   0   1   1   4]]
Score : 0.6135338345864662


## Logistic Regression

In [0]:
predict_and_plot_confusion_matrix(mean_embedded, y,
                                  SGDClassifier(class_weight='balanced', alpha=0.001, penalty='l2', loss='log', random_state=32 ,max_iter=1000, tol=0.0001)
                                 )

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Log loss : 1.342423864672542
Actual : 2
Probability : [[0.0591 0.2123 0.0315 0.1036 0.0482 0.0512 0.4884 0.0048 0.0008]]
Predicted : [7]
[[ 54   9   0  32   1   6  12   0   0]
 [  3  24   0   6   0   0  56   0   2]
 [  0   1   0   5   1   0  11   0   0]
 [ 31   5   0  85   0   0  16   0   0]
 [ 12   0   0  10  10   5  11   0   0]
 [ 11   2   0   7   1  19  15   0   0]
 [  2  19   0  10   0   0 160   0   0]
 [  1   0   0   0   0   0   3   0   0]
 [  0   1   0   3   0   0   1   0   2]]
Score : 0.5323308270676692


## Random Forest

In [0]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=1000, oob_score=True)
predict_and_plot_confusion_matrix(mean_embedded, y, clf)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Log loss : 1.052296765627609
Actual : 1
Probability : [[0.4246 0.1875 0.0156 0.142  0.0494 0.0375 0.1285 0.0082 0.0067]]
Predicted : [1]
[[ 65   1   1  29   4   1  13   0   0]
 [  3  46   0   2   1   1  38   0   0]
 [  1   0   8   1   0   0   8   0   0]
 [ 14   4   2 108   0   0   9   0   0]
 [ 14   1   1   8  14   1   9   0   0]
 [  8   2   0   2   1  29  13   0   0]
 [  2  19   1   4   1   0 164   0   0]
 [  2   0   0   1   0   0   1   0   0]
 [  2   0   0   0   0   0   1   0   4]]
Score : 0.6586466165413534


## SVC

In [0]:
clf = SGDClassifier(alpha=0.001, penalty='l2', loss='hinge', random_state=123 ,max_iter=10000, tol=0.00001)
predict_and_plot_confusion_matrix(mean_embedded, y, clf)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Log loss : 1.2724323439779115
Actual : 4
Probability : [[0.0547 0.1543 0.0209 0.2002 0.0388 0.0457 0.4816 0.0025 0.0013]]
Predicted : [7]
[[ 65   3   0  28   6   6   6   0   0]
 [  9  23   0   3   0   1  55   0   0]
 [  1   0   0   6   1   0  10   0   0]
 [ 32   1   0  76   7   2  19   0   0]
 [  8   2   0   6  18   2  12   0   0]
 [ 10   0   0   4   2  20  19   0   0]
 [ 12  22   0  11   0   1 145   0   0]
 [  0   0   0   1   0   0   2   0   1]
 [  1   0   0   2   0   0   2   0   2]]
Score : 0.524812030075188


## Perceptron

In [0]:
from sklearn.linear_model import Perceptron
clf = Perceptron(max_iter=10000, tol=0.00001)
predict_and_plot_confusion_matrix(mean_embedded, y, clf)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Log loss : 1.2978257211115216
Actual : 1
Probability : [[0.0925 0.0865 0.029  0.0853 0.1162 0.1096 0.4743 0.0054 0.0012]]
Predicted : [7]
[[ 42   0   0  50   2   7  12   0   1]
 [  7  15   0   2   1   2  64   0   0]
 [  3   0   0   4   2   0   9   0   0]
 [ 21   1   0  85   4   3  23   0   0]
 [ 12   1   0   7   6   5  17   0   0]
 [  6   1   0   6   4  22  16   0   0]
 [  6   3   0  14   0   0 168   0   0]
 [  1   0   0   2   0   0   1   0   0]
 [  0   0   0   2   0   0   1   0   4]]
Score : 0.5142857142857142


## Support Vector Classifier

In [0]:
from sklearn.svm import SVC
clf =  SVC(kernel="linear", C=1)
predict_and_plot_confusion_matrix(mean_embedded, y, clf)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Log loss : 1.9451888796706978
Actual : 4
Probability : [[3.154e-01 2.380e-02 2.000e-04 5.012e-01 5.580e-02 8.130e-02 2.230e-02
  0.000e+00 0.000e+00]]
Predicted : [4]
[[ 31   0   0  53   0   3  21   4   2]
 [  1   0   0   3   0   0  85   2   0]
 [  1   0   0   7   0   0  10   0   0]
 [ 18   0   0  82   0   4  29   4   0]
 [ 10   0   0  16   0   4  14   4   0]
 [  8   0   0  16   0  12  19   0   0]
 [  1   0   0   7   0   0 179   3   1]
 [  1   0   0   1   0   0   2   0   0]
 [  1   0   0   3   0   0   0   0   3]]
Score : 0.4616541353383459


## MLP Classifier ANN

In [0]:
clf = MLPClassifier(hidden_layer_sizes=(500,300,100), max_iter=1000, alpha=0.0001,solver='adam', verbose=10,  random_state=21, tol=0.000000001)
predict_and_plot_confusion_matrix(mean_embedded, y, clf)

Iteration 1, loss = 1.91939032
Iteration 2, loss = 1.84416537
Iteration 3, loss = 1.83593800
Iteration 4, loss = 1.83324555
Iteration 5, loss = 1.83354310
Iteration 6, loss = 1.82965069
Iteration 7, loss = 1.82777794
Iteration 8, loss = 1.82782871
Iteration 9, loss = 1.83082607
Iteration 10, loss = 1.81809141
Iteration 11, loss = 1.79468963
Iteration 12, loss = 1.84032221
Iteration 13, loss = 1.80641061
Iteration 14, loss = 1.78630206
Iteration 15, loss = 1.75900094
Iteration 16, loss = 1.71871381
Iteration 17, loss = 1.69393742
Iteration 18, loss = 1.63372993
Iteration 19, loss = 1.65303152
Iteration 20, loss = 1.62196200
Iteration 21, loss = 1.57473978
Iteration 22, loss = 1.59362802
Iteration 23, loss = 1.56938077
Iteration 24, loss = 1.58983416
Iteration 25, loss = 1.58046589
Iteration 26, loss = 1.58196848
Iteration 27, loss = 1.54165900
Iteration 28, loss = 1.58183448
Iteration 29, loss = 1.51948454
Iteration 30, loss = 1.52763779
Iteration 31, loss = 1.51273574
Iteration 32, los

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Iteration 1, loss = 1.96298350
Iteration 2, loss = 1.84515582
Iteration 3, loss = 1.84249095
Iteration 4, loss = 1.84171607
Iteration 5, loss = 1.83422421
Iteration 6, loss = 1.83435220
Iteration 7, loss = 1.83309429
Iteration 8, loss = 1.83197836
Iteration 9, loss = 1.84124827
Iteration 10, loss = 1.83070305
Iteration 11, loss = 1.83191197
Iteration 12, loss = 1.82586493
Iteration 13, loss = 1.82893655
Iteration 14, loss = 1.82535115
Iteration 15, loss = 1.82139229
Iteration 16, loss = 1.81681753
Iteration 17, loss = 1.81348841
Iteration 18, loss = 1.80149121
Iteration 19, loss = 1.79481927
Iteration 20, loss = 1.79950636
Iteration 21, loss = 1.79259800
Iteration 22, loss = 1.77923365
Iteration 23, loss = 1.77082636
Iteration 24, loss = 1.77149498
Iteration 25, loss = 1.81425608
Iteration 26, loss = 1.77815067
Iteration 27, loss = 1.72635600
Iteration 28, loss = 1.70472545
Iteration 29, loss = 1.67900451
Iteration 30, loss = 1.65356829
Iteration 31, loss = 1.68108129
Iteration 32, los

## Ensemble Model Extra Tree Classifier

In [0]:
from sklearn.ensemble.forest import ExtraTreesClassifier
clf = ExtraTreesClassifier( n_estimators= 1000,  max_features= None,bootstrap=True, oob_score= True, class_weight='balanced', n_jobs= -1) 
predict_and_plot_confusion_matrix(mean_embedded, y, clf)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Log loss : 0.9938384726993784
Actual : 1
Probability : [[0.3842 0.0615 0.0183 0.1592 0.237  0.0778 0.0518 0.005  0.0053]]
Predicted : [1]
[[ 69   2   1  18   9   4  11   0   0]
 [  4  43   0   1   0   1  42   0   0]
 [  1   0   7   3   0   0   7   0   0]
 [ 23   3   1 100   1   2   6   0   1]
 [ 13   2   1   6  19   3   4   0   0]
 [  5   0   0   2   0  39   9   0   0]
 [  4  11   3   3   2   0 168   0   0]
 [  0   0   0   0   0   0   2   1   1]
 [  1   0   0   0   0   0   0   1   5]]
Score : 0.6781954887218045
